# Data Science for Beginners - Part 2
Now that you have your environment running, we can do some stuff. We'll use this dataset:

[*2015 Flight Delays and Cancellations* provided by the U.S. Department of Transportation](https://www.kaggle.com/usdot/flight-delays). 

It is best to follow along with a running Jupyter environment - this can be local or on Azure Notebooks. It is also best to be running this on a cloned repo or library so that you'll also have the data accessible. 

**Note:** if on Azure notebooks - you would probably need to use a subset of the data because `flights.csv` may be too large for Azure to handle. In *Section 2*, run the other cell.

# 1. Import Modules and Data

In [1]:
%matplotlib inline

import pandas as pd # Panel data processing
import numpy as np # Library for numerical data
import matplotlib.pyplot as plt # Plots
import matplotlib 

# 2. Read Data from File

**We host this data on an S3 bucket, so it can take a while to download. You can choose to just download the file yourself and change the path.**

First we load our csv with `pd.read_csv()`, which is a very good csv parser. This returns a pandas `DataFrame` object, of which dimensions are accessible with `.shape`. 

`.head()` gives us a preview of our data- by showing the first 5 rows. 

In jupyter notebook, any variable called on the last line is automatically printed, so there's no need to call `print(flights.head())`.

### FOR AZURE NOTEBOOKS

In [3]:
flights = pd.read_csv('https://s3.amazonaws.com/vandyhacks/datascience/flights/flights_sample.csv') # Load the csv from our s3 bucket
# 80% sample of original data

print("Dimensions: {r} rows, {c} cols".format(r=flights.shape[0], c=flights.shape[1])) 
flights.head()

### FOR LOCAL/Other

In [6]:
flights = pd.read_csv('https://s3.amazonaws.com/vandyhacks/datascience/flights/flights.csv') # Load the csv from our s3 bucket
# This might take a while since the file is about 500mb big

print("Dimensions: {r} rows, {c} cols".format(r=flights.shape[0], c=flights.shape[1])) 
flights.head()

C:\Users\Kenneth\Miniconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Dimensions: 5819079 rows, 31 cols


,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,1,1,4,AS,98,N407AS,ANC,SEA,5,...,408.0,-22.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,...,741.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,2015,1,1,4,US,840,N171US,SFO,CLT,20,...,811.0,5.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,...,756.0,-9.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,2015,1,1,4,AS,135,N527AS,SEA,ANC,25,...,259.0,-21.0,0,0,NaN,NaN,NaN,NaN,NaN,NaN


## ALL notebooks run from here down

In [23]:
airlines = pd.read_csv('https://s3.amazonaws.com/vandyhacks/datascience/flights/airlines.csv')
airlines.head() 

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
